In [4]:
import os
import random
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
random.seed(datetime.now())

In [5]:
train_dir='../../exp_data'
train_jail_dir=os.path.join(train_dir,'jail')
vali_dir=os.path.join(train_dir,'vali')
labels=['negative','positive','polluted']
num_of_moving=[100,100,100]
prefix='otr_'

In [10]:
for i in range(3):
    print("Processing ",labels[i])
    filenames=[]
    for file in os.listdir(os.path.join(train_dir,labels[i])):
        if 'txt' not in file:
            filenames.append(file)
    filenames_set=set(filenames)
    filenames_set_bu=set(filenames)
    to_move_id=random.sample(range(0,len(filenames)),num_of_moving[i])
    to_moves=[filenames[id] for id in to_move_id]
    to_moves_group=[g[:g.find("_")] for g in to_moves]
    to_moves_group=list(set(to_moves_group))
    print("Size of tomove:",len(to_moves))
    print("Size of group:",len(to_moves_group))
    prison=0
    for to_move in to_moves:
        filenames_set.remove(to_move)
        filenames_set_bu.remove(to_move)
    for g in to_moves_group:
        if g!="p":
            for f in filenames_set_bu:
                if g in f:
                    #print("JAIL:",f,"(GROUP",g,")")
                    filenames_set.remove(f)
                    prison+=1
    print("Total Jail:",prison)
    

Processing  negative
Size of tomove: 100
Size of group: 96
Total Jail: 964
Processing  positive
Size of tomove: 100
Size of group: 40
Total Jail: 191
Processing  polluted
Size of tomove: 100
Size of group: 54
Total Jail: 399
